In [49]:
def cyk_algorithm(grammar, start_symbol, input_string):
    """
    Implementa el algoritmo CYK para una gramática en forma normal de Chomsky.

    Parameters:
        grammar (dict): Gramática libre de contexto en forma de diccionario,
                        donde las claves son símbolos no terminales y los valores
                        son listas de producciones.
        start_symbol (str): Símbolo inicial de la gramática.
        input_string (str): Cadena de entrada a analizar.
    """
    input_string = input_string.split()
    n = len(input_string)
    table = [[set() for _ in range(n)] for _ in range(n)]

    # Paso 1: Llenar la diagonal con las producciones que generan cada símbolo terminal
    for i in range(n):
        applied_rules_diagonal = []  # Reset for each terminal
        for lhs, rhs_list in grammar.items():
            for rhs in rhs_list:
                if len(rhs) == 1 and rhs[0] == input_string[i]:
                    table[i][i].add(lhs)
                    applied_rules_diagonal.append(f"{lhs} -> {rhs} (usando [{i},{i}])")

        print(f"Procesando el símbolo: {input_string[i]}")
        print("Estado actual de la matriz CYK:")
        print(f"{input_string}")
        display_matrix(table, n)

        if applied_rules_diagonal:
            print("Reglas aplicadas:")
            for rule in applied_rules_diagonal:
                print(f"  {rule}")
        else:
            print(f"No se encontraron reglas para el símbolo: {input_string[i]}")

        input("Presiona Enter para continuar...")

    # Paso 2: Llenar las otras celdas del triángulo superior
    for start in range(n - 2, -1, -1):
        for length in range(2, n - start + 1):
            end = start + length - 1
            applied_rules_non_diagonal = []  # Reset for each cell
            for split in range(start, end):
                left = table[start][split]
                right = table[split + 1][end]
                for lhs, rhs_list in grammar.items():
                    for rhs in rhs_list:
                        if len(rhs) == 2 and rhs[0] in left and rhs[1] in right:
                            table[start][end].add(lhs)
                            applied_rules_non_diagonal.append(
                                f"{lhs} -> {rhs} (usando [{start},{split}] y [{split+1},{end}])")

            print("Estado actual de la matriz CYK:")
            print(f"{input_string}")
            display_matrix(table, n)
            print(f"Procesando celda [{start}, {end}], longitud {length}")

            if applied_rules_non_diagonal:
                print("Reglas aplicadas:")
                for rule in applied_rules_non_diagonal:
                    print(f"  {rule}")
            else:
                print("No se aplicaron reglas en esta iteración.")

            input("Presiona Enter para continuar...")

    # Resultado
    print("Estado final de la matriz CYK:")
    display_matrix(table, n)
    if start_symbol in table[0][n - 1]:
        print("\nLa cadena pertenece al lenguaje.")
    else:
        print("\nLa cadena NO pertenece al lenguaje.")

    return table

def display_matrix(table, n):
    """Muestra la matriz CYK en una tabla legible."""
    for row in range(n):
        print(" ".join(["{" + ",".join(sorted(cell)) + "}" if cell else "{}" for cell in table[row]]))
    print()



# Ejemplo de uso
def main():
    # Gramática en forma normal de Chomsky
    grammar = {
        'O': [['SN', 'SV']],
        'SN': [['gato'], ['pescado'], ['perro'], ['pez']],
        'Det': [['el'], ['la'], ['una']],
        'SN': [['Det', 'N']],
        'N': [['gato'], ['pescado'], ['perro'], ['pez']],
        'SV': [['salta', 'duerme']],
        'Vt': [['ve'], ['ama'], ['come']],
        'SV': [['Vt', 'SN']],
    }
    start_symbol = 'O'
    input_string = "el gato come pescado"

    cyk_algorithm(grammar, start_symbol, input_string)

if __name__ == "__main__":
    main()


Procesando el símbolo: el
Estado actual de la matriz CYK:
['el', 'gato', 'come', 'pescado']
{Det} {} {} {}
{} {} {} {}
{} {} {} {}
{} {} {} {}

Reglas aplicadas:
  Det -> ['el'] (usando [0,0])
Presiona Enter para continuar...
Procesando el símbolo: gato
Estado actual de la matriz CYK:
['el', 'gato', 'come', 'pescado']
{Det} {} {} {}
{} {N} {} {}
{} {} {} {}
{} {} {} {}

Reglas aplicadas:
  N -> ['gato'] (usando [1,1])
Presiona Enter para continuar...
Procesando el símbolo: come
Estado actual de la matriz CYK:
['el', 'gato', 'come', 'pescado']
{Det} {} {} {}
{} {N} {} {}
{} {} {Vt} {}
{} {} {} {}

Reglas aplicadas:
  Vt -> ['come'] (usando [2,2])
Presiona Enter para continuar...
Procesando el símbolo: pescado
Estado actual de la matriz CYK:
['el', 'gato', 'come', 'pescado']
{Det} {} {} {}
{} {N} {} {}
{} {} {Vt} {}
{} {} {} {N}

Reglas aplicadas:
  N -> ['pescado'] (usando [3,3])
Presiona Enter para continuar...
Estado actual de la matriz CYK:
['el', 'gato', 'come', 'pescado']
{Det} {} 

In [3]:
import numpy as np
from IPython.display import display, clear_output

def cyk_algorithm(grammar, start_symbol, input_string):
    """
    Implementa el algoritmo CYK para una gramática en forma normal de Chomsky.

    Parameters:
        grammar (dict): Gramática libre de contexto en forma de diccionario,
                        donde las claves son símbolos no terminales y los valores
                        son listas de producciones.
        start_symbol (str): Símbolo inicial de la gramática.
        input_string (str): Cadena de entrada a analizar.
    """
    # Dividir la cadena de entrada en símbolos
    input_string = input_string.split()
    n = len(input_string)

    # Inicializar la matriz CYK (triangular superior)
    table = [[set() for _ in range(n)] for _ in range(n)]
    backpointers = [[{} for _ in range(n)] for _ in range(n)]  # Para construir el árbol

    # Paso 1: Llenar la diagonal con las producciones que generan cada símbolo terminal
    for j in range(n):
        applied_rules = []
        for lhs, rhs_list in grammar.items():
            for rhs in rhs_list:
                if len(rhs) == 1 and rhs[0] == input_string[j]:
                    table[j][j].add(lhs)
                    backpointers[j][j][lhs] = rhs[0]  # Guardar la regla aplicada
                    applied_rules.append(f"{lhs} -> {rhs}")
        #clear_output(wait=True)
        print("Estado actual de la matriz CYK:")
        display_matrix(table, n)
        print(f"Procesando celda [{j}, {j}] (símbolo: {input_string[j]})")
        if applied_rules:
            print("Reglas aplicadas:")
            for rule in applied_rules:
                print(f"  {rule}")
        else:
            print("No se aplicaron reglas en esta iteración.")
        input("Presiona Enter para continuar...")

    # Paso 2: Llenar las otras celdas del triángulo superior
    for length in range(2, n + 1):  # Longitud del segmento
        for start in range(n - length + 1):  # Índice inicial del segmento
            end = start + length - 1  # Índice final del segmento

            applied_rules = []
            for split in range(start, end):  # Punto de división
                left = table[start][split]
                right = table[split + 1][end]

                for lhs, rhs_list in grammar.items():
                    for rhs in rhs_list:
                        if len(rhs) == 2 and rhs[0] in left and rhs[1] in right:
                            table[start][end].add(lhs)
                            backpointers[start][end][lhs] = (split, rhs[0], rhs[1])  # Guardar divisiones
                            applied_rules.append(f"{lhs} -> {rhs} (usando [{start},{split}] y [{split+1},{end}])")

            #clear_output(wait=True)
            print("Estado actual de la matriz CYK:")
            display_matrix(table, n)
            print(f"Procesando celda [{start}, {end}], longitud {length}")
            if applied_rules:
                print("Reglas aplicadas:")
                for rule in applied_rules:
                    print(f"  {rule}")
            else:
                print("No se aplicaron reglas en esta iteración.")
            input("Presiona Enter para continuar...")

    # Resultado final
    #clear_output(wait=True)
    print("Estado final de la matriz CYK:")
    display_matrix(table, n)
    if start_symbol in table[0][n - 1]:
        print("\nLa cadena pertenece al lenguaje.")
        print("\nÁrbol sintáctico:")
        print_tree(backpointers, 0, n - 1, start_symbol, input_string)
    else:
        print("\nLa cadena NO pertenece al lenguaje.")

    return table

def display_matrix(table, n):
    """Muestra la matriz CYK en una tabla legible."""
    for row in range(n):
        print(" ".join(["{" + ",".join(sorted(cell)) + "}" if cell else "{}" for cell in table[row]]))
    print()

def print_tree(backpointers, start, end, symbol, input_string):
    """Construye y muestra el árbol sintáctico a partir de los punteros."""
    if start == end:  # Caso base: símbolo terminal
        print(f"{symbol} -> {input_string[start]}")
        return

    split, left_symbol, right_symbol = backpointers[start][end][symbol]
    print(f"{symbol} -> {left_symbol} {right_symbol}")
    print_tree(backpointers, start, split, left_symbol, input_string)
    print_tree(backpointers, split + 1, end, right_symbol, input_string)

# Ejemplo de uso
def main():
    # Gramática en forma normal de Chomsky
    grammar = {
        'O': [['SN', 'SV']],
        'SN': [['gato'], ['pescado'], ['perro'], ['pez'], ['Det', 'N']],
        'Det': [['el'], ['la'], ['una']],
        'N': [['gato'], ['pescado'], ['perro'], ['pez']],
        'SV': [['salta'], ['duerme'], ['Vt', 'SN']],
        'Vt': [['ve'], ['ama'], ['come']]
    }
    start_symbol = 'O'
    input_string = "el gato come pescado"

    cyk_algorithm(grammar, start_symbol, input_string)

if __name__ == "__main__":
    main()

Estado actual de la matriz CYK:
{Det} {} {} {}
{} {} {} {}
{} {} {} {}
{} {} {} {}

Procesando celda [0, 0] (símbolo: el)
Reglas aplicadas:
  Det -> ['el']
Presiona Enter para continuar...
Estado actual de la matriz CYK:
{Det} {} {} {}
{} {N,SN} {} {}
{} {} {} {}
{} {} {} {}

Procesando celda [1, 1] (símbolo: gato)
Reglas aplicadas:
  SN -> ['gato']
  N -> ['gato']
Presiona Enter para continuar...
Estado actual de la matriz CYK:
{Det} {} {} {}
{} {N,SN} {} {}
{} {} {Vt} {}
{} {} {} {}

Procesando celda [2, 2] (símbolo: come)
Reglas aplicadas:
  Vt -> ['come']
Presiona Enter para continuar...
Estado actual de la matriz CYK:
{Det} {} {} {}
{} {N,SN} {} {}
{} {} {Vt} {}
{} {} {} {N,SN}

Procesando celda [3, 3] (símbolo: pescado)
Reglas aplicadas:
  SN -> ['pescado']
  N -> ['pescado']
Presiona Enter para continuar...
Estado actual de la matriz CYK:
{Det} {SN} {} {}
{} {N,SN} {} {}
{} {} {Vt} {}
{} {} {} {N,SN}

Procesando celda [0, 1], longitud 2
Reglas aplicadas:
  SN -> ['Det', 'N'] (us